### Example 4.2.1.1: 1-D interpolation on equally spaced input points

In [1]:
#Setting Peridynamic parameters
class PDoperator_:
    def __init__(self, number_input_points = None, xmin = None, xmax = None, 
               number_output_points = None, xvec_org = None, yvec_org = None, 
               n1order = None, nsize = None, order = None):
        self.totnode_org = number_input_points
        self.totnode = number_output_points
        self.xmin = xmin
        self.xmax = xmax
        self.xvec_org = xvec_org 
        self.yvec_org = yvec_org
        self.n1order = n1order
        self.nsize = nsize
        self.order = order
    

In [2]:
#Function that generates grid
class GridGenerator:
    def __init__(init, PDoperator):
        
        xmin = PDoperator.xmin 
        xmax = PDoperator.xmax
        totnode = PDoperator.totnode
        
        xvec = []
        dvolume = []
        dx = (xmax - xmin)/ totnode
        for i in range(totnode):
            xvec.append(round(xmin + dx*i +dx/2.0,6))
            dvolume.append(round(dx*1*1,6))
        init.xvec = xvec
        init.dvolume = dvolume
    
    

In [3]:
#Function that creates lump volumes
def LumpVolumes(PDoperator, gridgenerator):
    
    totnode_org = PDoperator.totnode_org 
    totnode = PDoperator.totnode 
    xvec_org = PDoperator.xvec_org
    
    xvec = gridgenerator.xvec 
    dvolume = gridgenerator.dvolume
    
    
    dvolumes_org = [0] * totnode_org
    tiny = 1E-7
    rho = [0]*totnode_org
    dvolume_org = [0]*totnode_org
    for i in range(totnode):
        rhosum = 0.0
        for j in range(totnode_org):
            rho[j] = 1.0/((abs(xvec[i] - xvec_org[j]))**3 + tiny)
            rhosum = rhosum + rho[j]
        for j in range(totnode_org):
            dvolume_org[j] = dvolume_org[j] + (rho[j] / rhosum)*dvolume[i]
    rhosum = 0.0;
    for j in range(totnode_org):
        rhosum += dvolume_org[j]
    print("Total volume of output points         = ",totnode*dvolume[0] )
    print("Total volume of computed input points = ", rhosum)
    return dvolume_org

In [4]:
#Calculates size of Peridynamic operator
def getSize1D(n1order):
    nsize = n1order + 1
    return nsize

In [5]:
#Function that sets the order of the Differential Operators
def SetDiffOperators1D(nsize):
    order = list(range(0,nsize))
    return order

In [6]:
class NodeFamily:
    def __init__(self, xsi_order = None, xsivec = None, deltax = None):
        self.xsi_order = xsi_order
        self.xsivec = xsivec
        self.deltax = deltax

In [7]:
import pandas as pd
#Function that finds the node family for each Peridynamic node
def GenerateNodeFamily(xvec_org, xvec):
    
    xsi_order = pd.DataFrame({'node':xvec_org})
    xsivec = pd.DataFrame({'node':xvec_org})
    #xsivec = pd.DataFrame()
    
    #Ordering nodes
    for i in range(len(xvec)):
        aux = []
        for j in range(len(xvec_org)):
                aux.append(math.sqrt((xvec[i]-xvec_org[j])**2))
        xsi_order[str(xvec[i])] = aux
        xsivec[str(xvec[i])] = aux
    
    xsi_order.set_index('node', inplace=True)
    for i in xsi_order:
        xsi_order[str(i)] = xsi_order[str(i)].sort_values().keys()
    xsi_order_2 = pd.DataFrame({'node':xvec_org})
    for i in xsi_order:
        xsi_order_2[str(i)] = list(xsi_order[str(i)].sort_values().keys())
    
    #Calculating deltas (horizon) we are selecting the largest bond
    deltax = []
    xsivec = xsivec.drop(columns="node")
    for i in xsivec:
        deltax.append(max(list(xsivec[str(i)])))
    
    NodeFam = NodeFamily()
    xsi_order_2 = xsi_order_2.drop(columns="node")
    NodeFam.xsi_order = xsi_order_2
    NodeFam.xsivec = xsivec
    NodeFam.deltax = deltax
    
    #aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]
    return NodeFam;

In [8]:
#Function that calculates the Peridynamics weights
def calc_weights(NodeFam, xvec):
    tiny = 0.000001
    weights = pd.DataFrame() 

    k=0    
    for i in xvec:
        xsivec_column = NodeFam.xsivec[str(i)].to_numpy()
        aux = []
        for xsi_mag in xsivec_column:
            aux.append((NodeFam.deltax[k]/(xsi_mag+tiny))**2)
        weights[str(i)] = aux
        k = k + 1
        
    return weights

In [9]:
import numpy as np
#Calculate p_operator
def p_operator_1d(x0,nsize,dvolumes_org, weights):
    
    x0_array = x0.to_numpy()
    weights_array = weights.to_numpy()
    aux2 = np.zeros((nsize,nsize))
    node_mem = 0;
   
    for i in x0_array:
        aux = np.zeros((nsize,1))
        for j in range(nsize):
            aux[j] = i**j
            
        #aux2 +=np.matmul(aux,np.transpose(aux))
        #aux2 = aux2*dvolumes_org[node_mem]
        aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]*weights[node_mem]
        #aux2 +=np.matmul(aux,np.transpose(aux))
        node_mem = node_mem + 1
    
    return aux2

In [10]:
#Form Diff A Matrix
import math

def FormDiffAmat1D(nsize, xvec_org, xvec, dvolumes_org, NodeFam):
    #An A matrix must be created for each of the Peridynamic nodes
    DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
    xsivec = pd.DataFrame()
    xsivec_mags = pd.DataFrame()
    
    for i in range(len(xvec)):
        aux = []
        for j in range(len(xvec_org)):
                aux.append(xvec_org[j]-xvec[i])
        xsivec[str(xvec[i])] = aux
        
    #weights must be calculated here it is going to be one for each Peridynamic node i.e. 200
    weights = calc_weights(NodeFam, xvec)

    A_prism = np.zeros((len(xvec), nsize,nsize ))
    j=0
    for i in xvec:     
        p_matrix = p_operator_1d(xsivec[str(i)],nsize, dvolumes_org,weights[str(i)])
        A_prism[j,:,:] = p_matrix
        j = j +1

    return A_prism
    

In [11]:
def FormInvDiffAmat1D(DiffAmat1D, nsize):
    a,b,c = DiffAmat1D.shape #get dimensions of DiffAmat1D
    DiffAmatInv1D = np.zeros((a, nsize,nsize ))
    for i in range(a):
        DiffAmatInv1D[i] = np.linalg.inv(DiffAmat1D[i])

    return DiffAmatInv1D

In [12]:
def FormDiffBvec1D(DiffAmatInv1D,nsize):
    a,b,c = DiffAmatInv1D.shape #get dimensions of DiffAmatInv1D
    DiffBvec1D_prism = np.zeros((a, nsize))
    DiffBvec1D = np.array([1,0,0])
    for i in range(a): 
        DiffBvec1D_prism[i,:] = np.matmul(DiffBvec1D,DiffAmatInv1D[i])
    return DiffBvec1D_prism

In [13]:
import numpy as np
import math
#Function that calculates the 1D pass derivative operator
def Derivative_Operator_1D_PASS(PDoperator, gridgenerator, dvolumes_org):
    
    xvec = gridgenerator.xvec
    n1order = PDoperator.n1order
    PDoperator.nsize = getSize1D(n1order)
    PDoperator.order = SetDiffOperators1D(PDoperator.nsize)
    DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
    DiffAmatInv1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmatInv 2D matrix
    DiffBvec1D = [0] * PDoperator.nsize
    DiffAvec1D = [0] * PDoperator.nsize
    plist = [0] * PDoperator.nsize
    blist = [0] * PDoperator.nsize
    weight = [0] * PDoperator.nsize
    rcvec = [0] * PDoperator.nsize
    wgt = [0] * PDoperator.totnode_org
    ndfam = [0] * PDoperator.totnode_org
    
   
    nsize = PDoperator.nsize
    
    xvec_org = PDoperator.xvec_org
    #Here is where we should calculate A matrix, inverse of A matrix and B vector; for every Peridynamic node.
    
    
    #Generate node family
    NodeFam = GenerateNodeFamily(xvec_org, xvec)
    #Form A array of A Matrices
    DiffAmat1D = FormDiffAmat1D(nsize, xvec_org, xvec, dvolumes_org, NodeFam);    
    #Form Inverse of A Matrices
    DiffAmatInv1D = FormInvDiffAmat1D(DiffAmat1D, nsize)
    #Form B vector
    DiffBvec1D = FormDiffBvec1D(DiffAmatInv1D,nsize)
    #print("Interpolating for N=2")
    #for i in range(PDoperator.totnode):
        #GenerateNodeFamily_0
        
    
    return DiffBvec1D
    
    

In [14]:
#The PDoperator class constructor has the following inputs
#PDoperator_name = PDoperator(number_input_points, xmin, xmax, output_points, 
#                             xvec_org, yvec_org)
number_input_points = 10
xmin = 0
xmax = 9
number_output_points = 200
xvec_org = list(range(xmin,xmax+1))
yvec_org = [2, 2, 2, 2, 2, 3.2, 3.2, 3.2, 3.2, 3.2]
n1order = 2

PDoperator = PDoperator_(number_input_points, xmin, xmax, number_output_points, 
                        xvec_org, yvec_org, n1order)

#Generating the grid
gridgenerator = GridGenerator(PDoperator)

#Create lump volumes
dvolumes_org = LumpVolumes(PDoperator, gridgenerator)
#Calculating the 1D derivative operator
h = Derivative_Operator_1D_PASS(PDoperator, gridgenerator,dvolumes_org)



Total volume of output points         =  9.0
Total volume of computed input points =  9.0


In [15]:
#np.zeros((len(gridgenerator.xvec), PDoperator.nsize))

In [16]:
#h.to_excel("output.xlsx")

In [17]:
print(h)
DiffBvec1D = np.array([1,0,0]) 
print(DiffBvec1D)
#print(np.matmul(DiffBvec1D,h[1]))

[[ 1.53953155e-05  1.40870043e-04 -2.39521389e-05]
 [ 1.30046865e-04  3.38681164e-04 -6.13820287e-05]
 [ 3.37207334e-04  4.30832557e-04 -8.50883841e-05]
 [ 6.13076981e-04  4.26049408e-04 -9.55259078e-05]
 [ 9.33279342e-04  3.34667669e-04 -9.33782937e-05]
 [ 1.27314917e-03  1.68840404e-04 -7.96072075e-05]
 [ 1.60819334e-03 -5.73118082e-05 -5.55004483e-05]
 [ 1.91476065e-03 -3.27654476e-04 -2.27129939e-05]
 [ 2.17094324e-03 -6.24156829e-04  1.67080538e-05]
 [ 2.35770353e-03 -9.27239925e-04  6.03260963e-05]
 [ 2.46017476e-03 -1.21641623e-03  1.05358097e-04]
 [ 2.46902023e-03 -1.47125718e-03  1.48769927e-04]
 [ 2.38166573e-03 -1.67267611e-03  1.87431107e-04]
 [ 2.20316084e-03 -1.80444129e-03  2.18326731e-04]
 [ 1.94640430e-03 -1.85474798e-03  2.38810965e-04]
 [ 1.63151479e-03 -1.81760434e-03  2.46871702e-04]
 [ 1.28425451e-03 -1.69375415e-03  2.41364376e-04]
 [ 9.33603003e-04 -1.49089804e-03  2.22169906e-04]
 [ 6.08784327e-04 -1.22308937e-03  1.90240893e-04]
 [ 3.36202134e-04 -9.09348466e-

In [18]:
print(gridgenerator.xvec)
print(xvec_org[2]-gridgenerator.xvec[0])

[0.0225, 0.0675, 0.1125, 0.1575, 0.2025, 0.2475, 0.2925, 0.3375, 0.3825, 0.4275, 0.4725, 0.5175, 0.5625, 0.6075, 0.6525, 0.6975, 0.7425, 0.7875, 0.8325, 0.8775, 0.9225, 0.9675, 1.0125, 1.0575, 1.1025, 1.1475, 1.1925, 1.2375, 1.2825, 1.3275, 1.3725, 1.4175, 1.4625, 1.5075, 1.5525, 1.5975, 1.6425, 1.6875, 1.7325, 1.7775, 1.8225, 1.8675, 1.9125, 1.9575, 2.0025, 2.0475, 2.0925, 2.1375, 2.1825, 2.2275, 2.2725, 2.3175, 2.3625, 2.4075, 2.4525, 2.4975, 2.5425, 2.5875, 2.6325, 2.6775, 2.7225, 2.7675, 2.8125, 2.8575, 2.9025, 2.9475, 2.9925, 3.0375, 3.0825, 3.1275, 3.1725, 3.2175, 3.2625, 3.3075, 3.3525, 3.3975, 3.4425, 3.4875, 3.5325, 3.5775, 3.6225, 3.6675, 3.7125, 3.7575, 3.8025, 3.8475, 3.8925, 3.9375, 3.9825, 4.0275, 4.0725, 4.1175, 4.1625, 4.2075, 4.2525, 4.2975, 4.3425, 4.3875, 4.4325, 4.4775, 4.5225, 4.5675, 4.6125, 4.6575, 4.7025, 4.7475, 4.7925, 4.8375, 4.8825, 4.9275, 4.9725, 5.0175, 5.0625, 5.1075, 5.1525, 5.1975, 5.2425, 5.2875, 5.3325, 5.3775, 5.4225, 5.4675, 5.5125, 5.5575, 5.6025,

In [19]:
print('Hello World')

Hello World
